In [ ]:
!pip install ultralytics opencv-python scikit-image numpy scipy flask flask-restful

In [ ]:
# Cette cellule crée le fichier data.yaml pour YOLO
# Le fichier a déjà été créé dans datasets/data.yaml

# Contenu du fichier data.yaml:
# train: datasets/train
# val: datasets/val
# test: datasets/test
# nc: 15
# names: ["cat1","cat2","cat3","cat4","cat5","cat6","cat7","cat8","cat9","cat10","cat11","cat12","cat13","cat14","cat15"]

print("✅ Fichier data.yaml déjà créé dans datasets/data.yaml")

In [ ]:
from ultralytics import YOLO
import torch

# Vérifier si GPU disponible
print(f"🖥️ GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Charger le modèle pré-entrainé (nano = le plus léger)
model = YOLO("yolov8n.pt")

# Entraînement OPTIMISÉ pour RTX 2050 (4GB VRAM)
model.train(
    data="datasets/data.yaml",
    epochs=20,
    imgsz=480,          # Réduit de 640 → 480 (économise VRAM)
    batch=8,            # Réduit de 16 → 8 (évite Out of Memory)
    workers=4,          # Nombre de workers pour charger les données
    device=0,           # Utiliser le GPU (0 = première carte)
    amp=True,           # Mixed precision (économise VRAM)
    cache=True,         # Cache les images en RAM (plus rapide)
    name="yolo15cats"
)

# Évaluation
metrics = model.val()
print("✅ Métriques d'évaluation:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")

# Sauvegarder le modèle pour l'API
import shutil
shutil.copy("runs/detect/yolo15cats/weights/best.pt", "backend/data/models/yolo15cats.pt")
print("✅ Modèle exporté vers backend/data/models/yolo15cats.pt")